In [6]:
import tensorflow as tf

# 定义模型训练的部署
tf.app.flags.DEFINE_integer("trans_step", 0, "训练模型的步数")
tf.app.flags.DEFINE_string("model_dir", "C:\\Users\\Administrator\\Git\\CFturb\\Deep_Learning\\ckpt\\myregression", "模型保存的路径")

FLAGS = tf.app.flags.FlAGS


# 建立一个线性回归的模型
class MyLinearRegression(object):
    """实现一个线性回归的训练"""
    def __init__(self):
        # 自己设定的学习率，如果学习率过大，那损失不会减小反而会增大，一般 0~1
        self.learning_rate = 0.004
        pass
    
    def inputs(self):
        """获取需要训练的数据"""
        # 因为要进行张量运算（矩阵）所以这里特指值定义成二维的
        # x:[100, 1]; y_true  x * 0.7 + 0.8
        x_data = tf.random_normal(shape=[100, 1], mean=0.0, stddev=1.0, name='x_data')
        # 假设不知道这个矩阵
        y_true = tf.matmul(x_data, [[0.7]]) + 0.8
        return x_data, y_true

    def inference(self, feature):
        """根据数据特征值建立线性回归模型，feature是特征值"""
        # 先定义一个命名空间避免混乱
        with tf.variable_scope("linear_model"):
            # w,b : x[100, 1] * w + b = y_predict
            # 随机初始化权重和偏置，注意，权重和偏置必须使用tf.Variable变量OP去定义，因为只有Variable才能被梯度下降（模型）所训练
            # 权重w应该是[1, 1]形状的矩阵，因为它参与了矩阵的乘法
            self.weight = tf.Variable(tf.random_normal(shape=[1, 1], mean=0.0, stddev=1.0), name="weight")
            # 偏置是直接加的，所以应该是[1]的形状
            self.bias = tf.Variable(tf.random_normal(shape=[1], mean=0.0, stddev=1.0), name="bias")
            # 建立模型预测
            y_predict = tf.matmul(feature, self.weight) + self.bias
        return y_predict
    
    def loss(self, y_true, y_predict):
        """根据预测值和真实值求出均方误差"""
        # 定义一个命名空间
        # sum((y_true-y_predict)^2)mean()
        with tf.variable_scope("losses"):
            # 求出损失
            # tf.reduce_mean()：对列表中的数据求和之后求平均值
            loss = tf.reduce_mean(tf.square(y_true - y_predict))       
        return loss
    
    def sgd_op(self, loss):
        """利用梯度下降优化器去优化损失(优化模型参数)"""
        # 定义一个命名空间
        with tf.variable_scope("train_op"):
            train_op = tf.train.GradientDescentOptimizer(self.learning_rate).minimize(loss)
        return train_op
    
    def merge_summary(self, loss):
        """定义收集张量的函数"""
        # 收集一些单值变量
        tf.summary.scalar("losses", loss)
        # 收集高维度的张量值
        tf.summary.histogram("w", self.weight)
        tf.summary.histogram("b", self.bias)
        # 合并变量（OP）
        merged = tf.summary.merge_all()
        return merged
    
    
    def train(self):
        """用于训练的函数"""
        # 获取默认的图
        g = tf.get_default_graph()
        # 在默认的图中进行操作
        with g.as_default():
            # 进行训练
            # 1.获取数据
            x_data, y_true = self.inputs()
            # 2.利用模型得出预测结果
            y_predict = self.inference(x_data)
            # 3.损失计算
            loss = self.loss(y_true, y_predict)
            # 4.优化损失
            train_op = self.sgd_op(loss)
            
            # 收集要观察的张量值
            merged = self.merge_summary(loss)
            
            # 定义一个保存文件的 SaverOP
            saver = tf.train.Saver()
                       
            
            # 开启会话运行训练
            with tf.Session() as sess:
                # 初始化变量
                sess.run(tf.global_variables_initializer())
                
                # 创建events文件
                file_writer = tf.summary.FileWriter("C:\\Users\\Administrator\\Git\\CFturb\\Deep_Learning\\tensorboard\\", graph=sess.graph)
                
                # 打印模型没有训练时的初始化的参数
                print("模型初始化的参数权重：%f, 偏置为：%f" % (self.weight.eval(), self.bias.eval()))
                
                # 加载模型，从模型中导出与当前训练的模型的代码相同的OP操作， 覆盖原来的值
                ckpt = tf.train.latest_checkpoint("C:\\Users\\Administrator\\Git\\CFturb\\Deep_Learning\\ckpt\\myregression.ckpt")
                # 判断模型是否存在
                if ckpt:
                    saver.restore(sess, ckpt)
                
                # 第一次加载模型的参数
                print("第一次加载模型的参数权重：%f, 偏置为：%f" % (self.weight.eval(), self.bias.eval()))
                print("以当前加载的参数开始进行训练")
                
                
                """由于只训练一次所以没有什么效果，那我们让它训练多次"""
                for i in range(FLAGS.trans_step):
                    # 接下来调用运行
                    _, summary = sess.run([train_op, merged])
                    # 把summary添加到文件
                    file_writer.add_summary(summary, i)
                    # 再打印一遍优化之后的参数 
                    print("第%d次模型优化后的参数权重：%f, 偏置为：%f, 损失为：%f" % (i, self.weight.eval(), self.bias.eval(), loss.eval()))
                    
                    # 每隔100步保存一次模型
                    if i % 100 == 0:
                        # 要指定路径 + 名字
                        saver.save(sess, FLAGS.model_dir)


# 运行整个程序
if __name__ == '__main__':
    lr = MyLinearRegression()
    lr.train()

DuplicateFlagError: The flag 'model_dir' is defined twice. First from C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py, Second from C:\ProgramData\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py.  Description from first occurrence: C:\Users\Administrator\Git\CFturb\Deep_Learning\ckpt\myregression

### 实现线性回归
1. **学习率、步长和梯度爆炸**
    * 学习率不应该设置过大、0~1之间的数，如果过大，导致梯度爆炸（损失、参数优化成nan）
    * 学习率越大，达到最终比较好的效果步长越小
    * 学习率越小，达到最终比较好的效果步长越大
    * 一般选择较小的学习率
    
2. **在极端条件下，权重的值变得非常大，以至于溢出，导致`NaN`值。以下是梯度爆炸解决方案（深度神经网络当中更容易出现）**
    * 重新设计网络
    * 调整学习率
    * 使用梯度截断（在训练过程中检查和限制梯度的大小）
    * 使用激活函数
    
3. **模型要优化的参数必须选择`tf.Variable`定义，并且指定`trainable`参数为可训练**


4. **增加命名空间**

    使代码结构更加清晰，TensorBoard图结构清楚

### 增加变量显示

目的：在TensorBoard当中观察模型的参数、损失值等变量值的变化

1. 收集变量

    * `tf.summary.scalar(name=" ", tensor)`收集损失函数和准确率等单值变量，name为变量的名字，tensor为值
    
    * `tf.summary.histogram(name=" ",  tensor)`收集高维度的变化参数
    
    * `tf.summary.image(name=" ", tensor)`收集输入的图片张量能显示图片
    
2. 合并变量写入事件文件

    * `merge = tf.summary.merge_all()`
    
    * 运行合并：`summary = sess.run(merged)`，每次迭代都需要运行
    
    * 添加：`FileWriter.add_summary(summary, i)` i表示第几次的值

### 模型的保存与加载

* `tf.train.Saver(var_list=None, max_to_keep=5)`
    * 保存和加载模型（保存文件格式：checkpoint文件），保存要指定路径和要保存的会话
    
    * var_list：指定将要保存和还原的变量，它可以作为一个dict或一个列表传递
    
    * max_to_keep：指示要保留的最近检查点文件的最大数量。创建新文件时，会删除较旧的文件。如果无或0，则保留所有检查点文件，默认为5（即保留5个检查点文件。）
    

* `tf.Saver.restore(sess, '路径')`


* 保存：指定路径，**指定要保存的会话**，默认保存`tf.Variable`的OP，可以指定保存哪些

* 加载：本地文件的模型当中的一些变量名字与要进行加载训练的代码中的变量名字必须一致，代码训练以本地文件模型的参数值继续训练


## 仍有问题